In [ ]:
from imports_LDVAE_local import *
from scipy.stats import spearmanr
from matplotlib.colors import Normalize

In [ ]:
pre = "MH81"
drive = 'F'
base_model_path,base_table_path,base_plots_path,base_anndata_objects = h.return_local_paths(drive = drive,
                                                                              pre = pre,
                                                                              add_path = True)

In [ ]:
#load proof of principle results from MH80
_,base_table_path_MH80,_,_ = h.return_local_paths(drive = drive,pre = 'MH80',add_path = False)

In [ ]:
set_types = ['train','test']
celltypes = ['B',
 'CD14 Mono',
 'CD16 Mono',
 'CD4 T',
 'CD8 T',
 'MAIT',
 'NK+Proliferating']

In [ ]:
df_spearman = pd.DataFrame(columns = ['r_cyno_human','r_cyno_shifted_human','set_type','celltype'])
for set_type in set_types:
    for celltype in celltypes:
        df_temp = pd.read_csv(os.path.join(base_table_path_MH80,'MH80_df_ct_mean_expr_' + str(set_type) + '_' + str(celltype) + '.csv'),index_col=0).T

        df_temp['rank_00hr_cyno_'+ str(set_type) ] = df_temp['mean_expression_00hr_cyno_'+ str(set_type) ].rank(method='dense', ascending=False).astype(int)

        df_temp['rank_00hr_human_'+ str(set_type) ] = df_temp['mean_expression_00hr_human_'+ str(set_type) ].rank(method='dense', ascending=False).astype(int)

        df_temp['rank_00hr_cyno_shifted_'+ str(set_type) ] =  df_temp['mean_expression_00hr_cyno_shifted_'+ str(set_type) ].rank(method='dense', ascending=False).astype(int)

        r_cyno_human = spearmanr(df_temp['rank_00hr_cyno_'+ str(set_type)],df_temp['rank_00hr_human_'+ str(set_type)])[0]

        r_cyno_shifted_human = spearmanr(df_temp['rank_00hr_cyno_shifted_'+ str(set_type)],df_temp['rank_00hr_human_'+ str(set_type)])[0]
        df_spearman = df_spearman.append(pd.DataFrame([r_cyno_human,r_cyno_shifted_human,set_type,celltype],index = ['r_cyno_human','r_cyno_shifted_human','set_type','celltype']).T)  

In [ ]:
df_spearman['set_type+celltype'] = df_spearman['set_type'].astype(str) + '_' + df_spearman['celltype'].astype(str)

df_spearman['r_cyno_shifted_human'] = pd.to_numeric(df_spearman['r_cyno_shifted_human'],errors='coerce')

df_spearman['r_cyno_human'] = pd.to_numeric(df_spearman['r_cyno_human'], errors='coerce')

df_pivot_r_cyno_human = df_spearman.pivot(index='set_type', columns='celltype', values='r_cyno_human')
df_pivot_r_cyno_shifted_human = df_spearman.pivot(index='set_type', columns='celltype', values='r_cyno_shifted_human')

df_pivot_r_cyno_shifted_human = df_pivot_r_cyno_shifted_human.reindex(['train', 'test'])

df_pivot_r_cyno_human = df_pivot_r_cyno_human.reindex(['train', 'test'])

df_pivot_r_cyno_human = df_pivot_r_cyno_human.reindex(['train', 'test'])

df_all = df_pivot_r_cyno_human.append(df_pivot_r_cyno_shifted_human)

In [ ]:
df_all.index =['train_human_cyno', 'test_human_cyno', 'train_human_shifted_cyno', 'test_human_shifted_cyno']
df_all.to_csv(os.path.join(base_table_path,pre + '_df_all_spearman.csv'))

In [ ]:
plt.figure(figsize=(10, 6))
sns.heatmap(df_all, cmap="RdYlGn", annot=True, fmt=".2f",
           cbar_kws={'label': 'Spearman rank correlation'},
            norm=Normalize(vmin=0.1, vmax=1),
           linecolor='grey',
            linewidths=0.05)

plt.title('Rank correlation proof principle')
plt.yticks(rotation=0)
plt.savefig(os.path.join(base_plots_path,pre + '_heatmap_rank_correlation_annot.pdf'),bbox_inches='tight')
plt.show()

In [ ]:
df_all_only_test = df_all.T[['test_human_cyno','test_human_shifted_cyno']].T

In [ ]:
df_all.to_csv(os.path.join(base_table_path,pre + '_df_all_only_test_spearman.csv'))

In [ ]:
plt.figure(figsize=(10, 3))
sns.heatmap(df_all_only_test, cmap="PuBuGn", annot=True, fmt=".2f",
           cbar_kws={'label': 'Spearman rank correlation'},
            norm=Normalize(vmin=0.1, vmax=1),
           linecolor='grey',
            linewidths=0.05)

plt.title('Rank correlation proof principle')
plt.yticks(rotation=0)
plt.savefig(os.path.join(base_plots_path,pre + '_only_test_heatmap_rank_correlation_annot.pdf'),bbox_inches='tight')
plt.show()

In [ ]:
base_package_version_path = h.return_package_version_local_path(drive=drive)
h.save_package_versions(base_package_version_path,pre,do_print = True)